In [6]:
pip install bs4


Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
    Running setup.py install for bs4: started
    Running setup.py install for bs4: finished with status 'done'


You should consider upgrading via the 'c:\users\night\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [7]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\night\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
from urllib.request import urlretrieve
import time
from selenium import webdriver
import pandas as pd
import re

In [3]:
spot=pd.read_csv('전국관광지데이터.csv')
items = spot.poi_nm

# spot=pd.read_csv('전처리_광훈.csv')
# items = spot.title

c:\users\night\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3427: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
items = items[:12400]
# items = items[12400:24800]

# items[24800:]

# items[:12650]


In [32]:
items

0          호국전진사
1        상주옹기판매장
2         추월산관광지
3           쇄기농장
4            보타사
          ...   
12395        통일암
12396        범천사
12397        백련암
12398       태양정사
12399        봉용사
Name: poi_nm, Length: 12400, dtype: object

In [6]:
options=webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'
)
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36
options.add_argument('headless')


driver=webdriver.Chrome(chrome_options=options)  
time.sleep(3)



# items = ['백수해안도로']
# items = ['호국전진사','상주옹기판매장', '추월산관광지', '쇄기농장', '보타사', '안성팜랜드초원눈썰매장', '내장산단풍겨울빛축제','오대산선재길',
#         '덕유산국립공원시인의거리','의왕자연학습공원의왕레일파크' ]

rating_df = pd.DataFrame()
review_elem = pd.DataFrame()
img_elem = pd.DataFrame()
final_elem = pd.DataFrame()

count = 0
current = 0
goal = len(items)
# items = ['홍스시']
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = bs(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        img_list = soup.select('.list_photo > li > a')
        title = soup.select('div.inner_place > h2.tit_location')

        if len(img_list) != 0 :
            for i,v in enumerate(img_list):
                pattern = r"(?<=url\().*(?='\))"
                url = re.search(pattern, v["style"]).group(0)
                row = {'Spot_pk':current,"ItemID":item,"ImgURL":url[1:]}
                row = pd.DataFrame(row, index=[i])
                img_elem = img_elem.append(row, ignore_index=True)
        else:
            row = {'Spot_pk':current,"ItemID":item,"ImgURL":None}
            row = pd.DataFrame(row, index=[0])
            img_elem = img_elem.append(row, ignore_index=True)
            
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:

                    final_row = {'Spot_pk':current, "ItemID":title[0].text, "UserID":user_id[0].get('data-userid'),"Rating":rating[0].text[0], "review" : user_review[0].text, "Timestamp":timestamp[0].text}
                    final_row = pd.DataFrame(final_row, index=[i])
                    final_elem = final_elem.append(final_row, ignore_index = True)
                
                    

                except IndexError:
                    final_row = {'Spot_pk':current, "ItemID":title[0].text, "UserID":None,"Rating":rating[0].text[0], "review" : user_review[0].text, "Timestamp":timestamp[0].text}
                    final_row = pd.DataFrame(final_row, index=[i])
                    final_elem = final_elem.append(final_row, ignore_index = True)
               
                except IndexError:
                    final_row = {'Spot_pk':current, "ItemID":title[0].text, "UserID":None,"Rating":rating[0].text[0], "review" : None, "Timestamp":timestamp[0].text}
                    final_row = pd.DataFrame(final_row, index=[i])
                    final_elem = final_elem.append(final_row, ignore_index = True)



    
        else :
            print("리뷰가 없습니다")
#         print(img_elem)
        try:
            for i in range(2,11):
                time.sleep(2)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(2)
                html = driver.page_source
                soup = bs(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
#                             row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
#                             row = pd.DataFrame(row, index=[i])
#                             rating_df = rating_df.append(row,ignore_index=True)
#                             review_row = {"UserID":user_id[0].get('data-userid'), "ItemID" : item, "review" : user_review[0].text}
#                             review_row = pd.DataFrame(review_row, index=[i])
#                             review_elem = review_elem.append(review_row, ignore_index = True)
                            #
                            final_row = {'Spot_pk':current, "ItemID":title[0].text, "UserID":user_id[0].get('data-userid'),"Rating":rating[0].text[0], "review" : user_review[0].text, "Timestamp":timestamp[0].text}
                            final_row = pd.DataFrame(final_row, index=[i])
                            final_elem = final_elem.append(final_row, ignore_index = True)

#                             time.sleep(1)
                            
                        except IndexError:

                            final_row = {'Spot_pk':current, "ItemID":title[0].text, "UserID":None,"Rating":rating[0].text[0], "review" : user_review[0].text, "Timestamp":timestamp[0].text}
                            final_row = pd.DataFrame(final_row, index=[i])
                            final_elem = final_elem.append(final_row, ignore_index = True)

                        except IndexError:
                            final_row = {'Spot_pk':current, "ItemID":title[0].text, "UserID":None,"Rating":rating[0].text[0], "review" : None, "Timestamp":timestamp[0].text}
                            final_row = pd.DataFrame(final_row, index=[i])
                            final_elem = final_elem.append(final_row, ignore_index = True)



                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("장소 존재 x")

<ipython-input-6-d9f55d16f68f>:8: DeprecationWarning: use options instead of chrome_options
  driver=webdriver.Chrome(chrome_options=options)


진행상황 : 1/1
식당 존재


<ipython-input-6-d9f55d16f68f>:45: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1]) # 새 탭으로 이동


더 이상 리뷰 존재 X
기본 페이지로 돌아가자


<ipython-input-6-d9f55d16f68f>:161: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])


In [7]:
final_elem

,Spot_pk,ItemID,UserID,Rating,review,Timestamp
0,1,홍스시 운암점,143lma7,2,수정) 맛있고 좋았었으나 주방에서 서빙직원들에게 짜증내는것이 너무나 잦으심 그게 식...,2021.07.31.
1,1,홍스시 운암점,aoh6vb,1,참치초밥 포장해먹다가 너무 이상해서 잠시만 떼 먹다가 그것마저 못먹고 다 버렸어요....,2021.07.26.
2,1,홍스시 운암점,no33jb,5,핵 존 맛.,2021.05.20.
3,1,홍스시 운암점,beoki1,5,,2021.03.11.
4,1,홍스시 운암점,vmd5tp,5,다른지역으로 이사갔는데 이만한 초밥집이 없어서 계속 생각나는 초밥집이에요 ! 광주로...,2020.08.15.
5,1,홍스시 운암점,k9bgcs,2,"초밥을 좋아하는 사람입니다. 오늘 4번째 방문을 했는데, 처음 나온 회덥밥은 goo...",2020.08.15.
6,1,홍스시 운암점,uq0goe,1,"1인과 2인, 2인과 3인 매뉴가 양이 똑같다는게 이해가 안가네요 2인은 59000...",2019.12.14.
7,1,홍스시 운암점,g938rv,1,,2019.11.05.
8,1,홍스시 운암점,7kjo63,1,스페셜 먹었습니다. 모듬스시가 먼저 나왔는데 푸석거립니다. 한점 크기도 커서 가위...,2019.04.20.
9,1,홍스시 운암점,h0p700,5,생선의 질이 너무나도 좋았습니다. 사장님께서 서비스로 가지구이초밥을 주셨는데 환상적...,2018.12.27.


In [8]:
img_elem

,Spot_pk,ItemID,ImgURL
0,1,홍스시,//img1.kakaocdn.net/relay/local/R640x320/?fnam...
1,1,홍스시,//img1.kakaocdn.net/relay/local/R640x320/?fnam...
2,1,홍스시,//img1.kakaocdn.net/relay/local/R640x320/?fnam...
3,1,홍스시,//img1.kakaocdn.net/relay/local/R640x320/?fnam...
4,1,홍스시,//img1.kakaocdn.net/relay/local/R640x320/?fnam...


In [ ]:
#저장
final_elem.to_csv('review_data.csv',encoding='euc-kr',index=False)
img_elem.to_csv('img_data.csv',encoding='euc-kr',index=False)